In [1]:
!pip install unidecode
!pip install investpy
!pip install plotly_express -U
!pip install plotly==4.14.3
!pip install unidecode

     |████████████████████████████████| 245kB 5.6MB/s 
     |████████████████████████████████| 4.4MB 5.8MB/s 
     |████████████████████████████████| 5.5MB 46.0MB/s 
     |████████████████████████████████| 512kB 41.0MB/s 
  Created wheel for investpy: filename=investpy-1.0.3-cp37-none-any.whl size=4466737 sha256=dddde8472dace9b917933eaa95bab29fccd3f24345909838f3283dc43cf9e680
  Stored in directory: /root/.cache/pip/wheels/ef/ef/31/cd22f970a59025c6d6dee56091b1f538d850e932697db1950e
Successfully built investpy
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
  Found existing installation: pytz 2018.9
    Uninstalling pytz-2018.9:
      Successfully uninstalled pytz-2018.9


     |████████████████████████████████| 13.2MB 294kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import urllib
import pandas as pd, datetime as dt
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from unidecode import unidecode

from IPython.display import display, HTML, display_html

root_path = os.getcwd()
save_path = os.path.join(os.getcwd(), root_path + "/_includes/")

In [3]:
!mkdir "/content/_includes/"

In [4]:
#Predição do encoding do arquivo
def predict_encoding(file_path, n_lines=20):
    '''Predict a file's encoding using chardet'''
    import chardet
    with open(file_path, 'rb') as f:
        rawdata = b''.join([f.readline() for _ in range(n_lines)])

    return chardet.detect(rawdata)['encoding']

def remove_acento(palavra):
    return(unidecode(palavra))

def ordem_faixa_etaria(name):
    #name = name.lower()
    if name == 'De 0 a 9': return 1
    if name == 'De 10 a 19': return 2
    if name == 'De 20 a 29': return 3
    if name == 'De 30 a 39': return 4
    if name == 'De 40 a 49': return 5
    if name == 'De 50 a 59': return 6
    if name == 'De 60 a 69': return 7
    if name == 'De 70 a 79': return 8
    if name == 'De 80 a 89': return 9
    if name == 'De 90 a 100': return 10
    if name == 'De 100 a 109': return 11   

In [5]:
#Populacao da cidade do Rio de Janeiro - fonte: Data.rio
remote_file_name = "http://pcrj.maps.arcgis.com/sharing/rest/content/items/35e67befce5e4519b95cba07e4d308e9/data"
populacao_file = os.path.join("populacao_rj.xls")
urllib.request.urlretrieve(remote_file_name, populacao_file) 

('populacao_rj.xls', <http.client.HTTPMessage at 0x7fa8a3c6b4d0>)

In [6]:
df_populacao = pd.read_excel(populacao_file, header=3)

In [7]:
df_populacao.columns = ['bairro','populacao','domicilios']
df_populacao['bairro'] = df_populacao['bairro'].str.upper()
df_populacao.drop(df_populacao[df_populacao['bairro'] == 'TOTAL'].index, inplace=True)
df_populacao.dropna(inplace=True)
df_populacao['bairro'] = df_populacao['bairro'].apply(remove_acento)
df_populacao = df_populacao.sort_values('populacao', ascending=False).set_index('bairro')

In [8]:
#df_populacao['populacao'] / df_populacao['domicilios']
df_populacao['moradores_domicilio'] = round(df_populacao['populacao'] / df_populacao['domicilios'],2)
#df_populacao.sort_values('moradores_domicilio', ascending=False).head(40)
#--df_populacao.moradores_domicilio.mean()

In [9]:
#Dataset covid-19 da cidade do Rio de Janeiro - fonte: Data.rio
remote_file_name = "http://pcrj.maps.arcgis.com/sharing/rest/content/items/ae85fc84a9b244108d96c7072be4d3d3/data"
remote_file_name = "http://pcrj.maps.arcgis.com/sharing/rest/content/items/ae85fc84a9b244108d96c7072be4d3d3/data"
covid_file = os.path.join("covid19_rj.csv")
urllib.request.urlretrieve(remote_file_name, covid_file) 

('covid19_rj.csv', <http.client.HTTPMessage at 0x7fa8a374f250>)

In [10]:
df_covid = pd.read_csv(covid_file,sep=';',encoding=predict_encoding(covid_file))

In [11]:
print(df_covid.shape, df_populacao.shape)

(223935, 12) (162, 3)


In [12]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223935 entries, 0 to 223934
Data columns (total 12 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   classificação_final    223935 non-null  object
 1   dt_notific             223885 non-null  object
 2   dt_inicio_sintomas     223824 non-null  object
 3   bairro_resid_estadia   223935 non-null  object
 4   ap_residencia_estadia  223935 non-null  object
 5   sexo                   223935 non-null  object
 6   faixa_etaria           223935 non-null  object
 7   evolucao               223935 non-null  object
 8   dt_evolucao            195239 non-null  object
 9   raca_cor               223935 non-null  object
 10  Data_atualizacao       223935 non-null  object
 11  sistema                223935 non-null  object
dtypes: object(12)
memory usage: 20.5+ MB


In [13]:
#redefinindo tamanho da área para trabalho
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', df_covid.shape[1]+3)

In [14]:
# Renomeando as colunas para melhor entendimento
df_covid.columns = ['classificacao', 'dt_notificacao', 'dt_inicio_sintomas','bairro', 'area_planejamento', 
              'sexo', 'faixa_etaria', 'evolucao', 'dt_obito', 'raca_cor', 'data_atualizacao','sistema']

In [15]:
print("Última atualização: {}".format(df_covid.data_atualizacao.describe()[2]))
ate = df_covid.data_atualizacao.describe()[2]#.strftime("%d/%m")

Última atualização: 03/29/21


In [16]:
#Limpeza e padronização de dados inválidos
df_covid = df_covid.replace('Missing', np.NaN)
df_covid = df_covid.replace('Ignorado','INDEFINIDO')
df_covid = df_covid.replace('N/D','INDEFINIDO')
df_covid = df_covid.replace('#N/D','INDEFINIDO')
df_covid = df_covid.replace('ND','INDEFINIDO')
df_covid = df_covid.replace('óbito','obito')

In [17]:
df_covid

,classificacao,dt_notificacao,dt_inicio_sintomas,bairro,area_planejamento,sexo,faixa_etaria,evolucao,dt_obito,raca_cor,data_atualizacao,sistema
0,CONFIRMADO,09/18/20,09/03/20,PACIENCIA,5.3,M,De 50 a 59,OBITO,09/18/20,Preta,03/29/21,SIVEP
1,CONFIRMADO,11/25/20,11/02/20,BARRA DA TIJUCA,4.0,M,De 80 a 89,OBITO,11/26/20,Branca,03/29/21,SIVEP
2,CONFIRMADO,05/06/20,05/06/20,CACHAMBI,3.2,M,De 70 a 79,OBITO,05/26/20,INDEFINIDO,03/29/21,SIVEP
3,CONFIRMADO,11/12/20,11/02/20,BARRA DA TIJUCA,4.0,M,De 70 a 79,OBITO,11/23/20,Branca,03/29/21,SIVEP
4,CONFIRMADO,06/13/20,04/26/20,MARECHAL HERMES,3.3,M,De 60 a 69,OBITO,06/23/20,INDEFINIDO,03/29/21,SIVEP
...,...,...,...,...,...,...,...,...,...,...,...,...
223930,CONFIRMADO,12/21/20,12/15/20,FORA DO MUNICIPIO,INDEFINIDO,F,De 20 a 29,RECUPERADO,12/29/20,Parda,03/29/21,ESUS
223931,CONFIRMADO,03/28/21,03/22/21,SAO CONRADO,2.1,F,De 70 a 79,ATIVO,NaN,Parda,03/29/21,ESUS
223932,CONFIRMADO,03/28/21,03/21/21,HONORIO GURGEL,3.3,M,De 70 a 79,ATIVO,NaN,Branca,03/29/21,ESUS
223933,CONFIRMADO,03/29/21,03/06/21,VILA DA PENHA,3.3,F,De 20 a 29,RECUPERADO,03/20/21,Branca,03/29/21,ESUS


In [18]:
pd.to_datetime(df_covid['dt_obito'], format='%d/%m/%Y', infer_datetime_format=True)

0        2020-09-18
1        2020-11-26
2        2020-05-26
3        2020-11-23
4        2020-06-23
            ...    
223930   2020-12-29
223931          NaT
223932          NaT
223933   2021-03-20
223934          NaT
Name: dt_obito, Length: 223935, dtype: datetime64[ns]

In [19]:
#Conversão colunas datas
df_covid['dt_notificacao'] =  pd.to_datetime(df_covid['dt_notificacao'], format='%d/%m/%Y', infer_datetime_format=True)
df_covid['dt_inicio_sintomas'] =  pd.to_datetime(df_covid['dt_inicio_sintomas'], format='%d/%m/%Y', infer_datetime_format=True)
df_covid['dt_obito'] =  pd.to_datetime(df_covid['dt_obito'], format='%d/%m/%Y', infer_datetime_format=True)
df_covid['data_atualizacao'] =  pd.to_datetime(df_covid['data_atualizacao'], format='%d/%m/%Y', infer_datetime_format=True)

In [20]:
#dataset população e covid19
display(df_populacao.tail(), df_covid.head())

,populacao,domicilios,moradores_domicilio
bairro,,,
CAMORIM,1970.0,659.0,2.99
CIDADE UNIVERSITARIA,1556.0,534.0,2.91
CAMPO DOS AFONSOS,1365.0,375.0,3.64
JOA,818.0,251.0,3.26
GRUMARI,167.0,44.0,3.80


,classificacao,dt_notificacao,dt_inicio_sintomas,bairro,area_planejamento,sexo,faixa_etaria,evolucao,dt_obito,raca_cor,data_atualizacao,sistema
0,CONFIRMADO,2020-09-18,2020-09-03,PACIENCIA,5.3,M,De 50 a 59,OBITO,2020-09-18,Preta,2021-03-29,SIVEP
1,CONFIRMADO,2020-11-25,2020-11-02,BARRA DA TIJUCA,4.0,M,De 80 a 89,OBITO,2020-11-26,Branca,2021-03-29,SIVEP
2,CONFIRMADO,2020-05-06,2020-05-06,CACHAMBI,3.2,M,De 70 a 79,OBITO,2020-05-26,INDEFINIDO,2021-03-29,SIVEP
3,CONFIRMADO,2020-11-12,2020-11-02,BARRA DA TIJUCA,4.0,M,De 70 a 79,OBITO,2020-11-23,Branca,2021-03-29,SIVEP
4,CONFIRMADO,2020-06-13,2020-04-26,MARECHAL HERMES,3.3,M,De 60 a 69,OBITO,2020-06-23,INDEFINIDO,2021-03-29,SIVEP


# Resumo geral

In [21]:
obitos  = df_covid[df_covid['evolucao'] == 'obito'].classificacao.count()
ativos  = df_covid[df_covid['evolucao'] == 'ativo'].classificacao.count()
recuperados  = df_covid[df_covid['evolucao'] == 'recuperado'].classificacao.count()
total = obitos + ativos + recuperados

In [22]:
fig = go.Figure(data=[go.Table(header=dict(values=['Última atualização', 'Total de casos', 'Recuperados', 'Ativos', 'Óbitos' ]),
                 cells=dict(values=[ate, total, recuperados, ativos, obitos ]))
                     ])
fig.update_layout(
    autosize=True,
    height=100,
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=10,
        pad=0
    ),
)

fig.show()
fig.write_html(os.path.join(save_path, 'resumo-geral.html'))

In [23]:
# Merge dadasets populacao 2 covid-19

In [24]:
df = None
if df is not None:
    del df
    
df = pd.merge(df_covid, df_populacao, how='left', on='bairro')

# Casos por bairro

In [25]:
df_bairro = df.groupby(['bairro','evolucao']).classificacao.count().reset_index().\
pivot_table(index=['bairro'],columns=['evolucao']).reset_index()
df_bairro.columns = ['bairro','ativo','obito','recuperado']
df_bairro['recuperado'] = df_bairro.recuperado.fillna(0)
df_bairro['ativo'] = df_bairro.ativo.fillna(0)
df_bairro['obito'] = df_bairro.obito.fillna(0)
df_bairro['total'] = df_bairro.ativo + df_bairro.recuperado + df_bairro.obito
df_bairro['total_acumulado'] = df_bairro.total.cumsum()

df_bairro = pd.merge(df_bairro, df_populacao, how='left', on='bairro')
df_bairro = df_bairro.drop(['domicilios', 'moradores_domicilio'], axis=1)
df_bairro.dropna(inplace=True)

print("Taxa de letalidade em {}: {}".format(ate, (df_bairro.sum().obito / df_bairro.sum().total) * 100))
print("Taxa de recuperação em {}: {}".format(ate, (df_bairro.sum().recuperado / df_bairro.sum().total) * 100))

Taxa de letalidade em 03/29/21: 9.414365501760264
Taxa de recuperação em 03/29/21: 84.72588098379974


In [26]:
df[df.dt_notificacao.isnull()]

,classificacao,dt_notificacao,dt_inicio_sintomas,bairro,area_planejamento,sexo,faixa_etaria,evolucao,dt_obito,raca_cor,data_atualizacao,sistema,populacao,domicilios,moradores_domicilio
50601,CONFIRMADO,NaT,2021-01-09,PENHA,3.1,M,De 50 a 59,RECUPERADO,2021-01-23,Preta,2021-03-29,ESUS,78678.0,24853.0,3.17
54533,CONFIRMADO,NaT,2020-12-21,AGUA SANTA,3.2,M,De 70 a 79,RECUPERADO,2021-01-04,INDEFINIDO,2021-03-29,ESUS,8756.0,3660.0,2.39
56230,CONFIRMADO,NaT,2020-12-31,JACAREPAGUA,4.0,F,De 10 a 19,RECUPERADO,2021-01-14,INDEFINIDO,2021-03-29,ESUS,157326.0,53500.0,2.94
56241,CONFIRMADO,NaT,2020-12-31,MADUREIRA,3.3,M,De 20 a 29,RECUPERADO,2021-01-14,INDEFINIDO,2021-03-29,ESUS,50106.0,16926.0,2.96
56339,CONFIRMADO,NaT,2021-01-09,GALEAO,3.1,F,De 40 a 49,RECUPERADO,2021-01-23,Branca,2021-03-29,ESUS,22971.0,7398.0,3.11
61068,CONFIRMADO,NaT,2021-01-21,TAQUARA,4.0,M,De 50 a 59,RECUPERADO,2021-02-04,Parda,2021-03-29,ESUS,102126.0,34209.0,2.99
61083,CONFIRMADO,NaT,2021-01-13,RECREIO DOS BANDEIRANTES,4.0,M,De 30 a 39,RECUPERADO,2021-01-27,Branca,2021-03-29,ESUS,84224.0,29118.0,2.89
67328,CONFIRMADO,NaT,2020-11-27,VILA ISABEL,2.2,M,De 50 a 59,RECUPERADO,2020-12-11,Parda,2021-03-29,ESUS,86018.0,31335.0,2.75
76991,CONFIRMADO,NaT,2020-11-25,PRACA SECA,4.0,F,De 40 a 49,RECUPERADO,2020-12-09,Parda,2021-03-29,ESUS,64147.0,21570.0,2.97
91051,CONFIRMADO,NaT,2020-12-08,INDEFINIDO,INDEFINIDO,M,De 20 a 29,RECUPERADO,2020-12-22,INDEFINIDO,2021-03-29,ESUS,NaN,NaN,NaN


In [27]:
# Se data notificação inexistente vou atualizar com a data de inicio dos sintomas, se essa tambem for inexistente
# vou atualizar com a data de atualizacao
df['dt_notificacao'].loc[df.dt_notificacao.isnull()] = df['dt_inicio_sintomas'].loc[df.dt_notificacao.isnull()]
df['dt_notificacao'].loc[df.dt_notificacao.isnull()] = df['data_atualizacao'].loc[df.dt_notificacao.isnull()]
# Inlcuir Coluna semana
df['semana'] = df.apply(lambda row: row['dt_notificacao'] - dt.timedelta(days=row['dt_notificacao'].weekday()), axis=1)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Distribuição por bairros

In [28]:
df_bairro  = df.groupby(['bairro','evolucao']).classificacao.count().reset_index().\
pivot_table(index=['bairro'],columns=['evolucao']).reset_index()

df_bairro.columns = ['bairro','ativo','obito','recuperado']
df_bairro['recuperado'] = df_bairro.recuperado.fillna(0).astype(int)
df_bairro['ativo'] = df_bairro.ativo.fillna(0).astype(int)
df_bairro['obito'] = df_bairro.obito.fillna(0).astype(int)
df_bairro['total'] = df_bairro.ativo + df_bairro.recuperado + df_bairro.obito

df_bairro = pd.merge(df_bairro, df_populacao, how='left', on='bairro')
df_bairro = df_bairro.drop(['domicilios', 'moradores_domicilio'], axis=1)
df_bairro.dropna(inplace=True)

df_bairro['taxa_letalidade'] = (df_bairro.obito / df_bairro.total) * 100
df_bairro['taxa_recuperacao'] = (df_bairro.recuperado / df_bairro.total) * 100
df_bairro['obito_1000_habitantes'] = round((df_bairro.obito * 1000)/ df_bairro.populacao).astype(int)
df_bairro['casos_1000_habitantes'] = round((df_bairro.total * 1000)/ df_bairro.populacao).astype(int)

df_bairro['populacao'] = df_bairro.populacao.astype(int)

print("Taxa de letalidade em {}: {}".format(ate, (df_bairro.sum().obito / df_bairro.sum().total) * 100))
print("Taxa de recuperação em {}: {}".format(ate, (df_bairro.sum().recuperado / df_bairro.sum().total) * 100))
df_bairro = df_bairro.set_index('bairro')

Taxa de letalidade em 03/29/21: 9.414365501760264
Taxa de recuperação em 03/29/21: 84.72588098379974


In [29]:
df_bairro.info()

<class 'pandas.core.frame.DataFrame'>
Index: 157 entries, ABOLICAO to ZUMBI
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ativo                  157 non-null    int64  
 1   obito                  157 non-null    int64  
 2   recuperado             157 non-null    int64  
 3   total                  157 non-null    int64  
 4   populacao              157 non-null    int64  
 5   taxa_letalidade        157 non-null    float64
 6   taxa_recuperacao       157 non-null    float64
 7   obito_1000_habitantes  157 non-null    int64  
 8   casos_1000_habitantes  157 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 12.3+ KB


In [30]:
def display_side_by_side(dfs:list, captions:list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))

In [31]:
display_side_by_side([df_bairro[['total','populacao']].sort_values('total', ascending=False)[:30],
                     df_bairro[['total','populacao','casos_1000_habitantes']].sort_values('casos_1000_habitantes', ascending=False)[:30]],
                    ['Casos  totais - Top20 bairros', 'Casos por 1000 habitantes - Top20 bairros'])

#fig.show()
#fig.write_html(os.path.join(save_path, 'casos-por-habitantes-tabela.html'))

,total,populacao
bairro,,
BARRA DA TIJUCA,9889,136831
TIJUCA,8859,163805
COPACABANA,8078,146392
CAMPO GRANDE,7805,328370
SANTA CRUZ,5884,217333
BOTAFOGO,5292,82890
RECREIO DOS BANDEIRANTES,5209,84224
JACAREPAGUA,5017,157326
BANGU,4808,220552


In [32]:
display_side_by_side([df_bairro[['obito','populacao']].sort_values('obito', ascending=False)[:50],df_bairro[['obito','populacao','obito_1000_habitantes']].sort_values('obito_1000_habitantes', ascending=False)[:50]],
                    ['Óbitos totais - Top20 bairros', 'Óbitos por 1000 habitantes - Top20 bairros'])

,obito,populacao
bairro,,
CAMPO GRANDE,1170,328370
BANGU,832,220552
TIJUCA,704,163805
COPACABANA,675,146392
REALENGO,623,180123
SANTA CRUZ,618,217333
BARRA DA TIJUCA,457,136831
TAQUARA,432,102126
IRAJA,325,96382


## Casos por mil habitantes - Top10 Bairros

In [33]:
df_bairro_top10 = df_bairro.sort_values('total', ascending=False)[:10]
fig = px.scatter(df_bairro_top10, x="populacao", y="total", size="obito", color=df_bairro_top10.index,
                hover_name=df_bairro_top10.index, log_x=False, size_max=60,
                template="simple_white",
                title="Casos por habitantes - Top10 Bairros")

fig.update_yaxes(showticklabels=True, visible=False)
fig.update_xaxes(showticklabels=True, visible=True, title='')

fig.show()
fig.write_html(os.path.join(save_path, 'casos-por-habitantes.html'))

In [34]:
df_bairro_diario  = df.groupby(['bairro','semana','evolucao']).classificacao.count().reset_index().\
pivot_table(index=['bairro','semana'], columns=['evolucao']).reset_index()

df_bairro_diario.columns = ['bairro','data','ativo','obito','recuperado']
df_bairro_diario['recuperado'] = df_bairro_diario.recuperado.fillna(0)
df_bairro_diario['ativo'] = df_bairro_diario.ativo.fillna(0)
df_bairro_diario['obito'] = df_bairro_diario.obito.fillna(0)
df_bairro_diario['total'] = df_bairro_diario.ativo + df_bairro_diario.recuperado + df_bairro_diario.obito

df_bairro_diario = pd.merge(df_bairro_diario, df_populacao, how='left', on='bairro')
df_bairro_diario = df_bairro_diario.drop(['domicilios', 'moradores_domicilio'], axis=1)
df_bairro_diario.dropna(inplace=True)

df_bairro_diario['taxa_letalidade'] = (df_bairro_diario.obito / df_bairro_diario.total) * 100
df_bairro_diario['taxa_recuperacao'] = (df_bairro_diario.recuperado / df_bairro_diario.total) * 100
df_bairro_diario['obito_1000_habitantes'] = ((df_bairro_diario.obito * 1000)/ df_bairro_diario.populacao)
df_bairro_diario['casos_1000_habitantes'] = ((df_bairro_diario.total * 1000)/ df_bairro_diario.populacao)

print("Taxa de letalidade em {}: {}".format(ate, (df_bairro_diario.sum().obito / df_bairro_diario.sum().total) * 100))
print("Taxa de recuperação em {}: {}".format(ate, (df_bairro_diario.sum().recuperado / df_bairro_diario.sum().total) * 100))
df_bairro_diario = df_bairro_diario.set_index('data')

df_bairro_diario[['obito','populacao','obito_1000_habitantes']].sort_values('obito_1000_habitantes', ascending=False)[:10]

df_bairro_diario.head()

Taxa de letalidade em 03/29/21: 9.414365501760264
Taxa de recuperação em 03/29/21: 84.72588098379974


,bairro,ativo,obito,recuperado,total,populacao,taxa_letalidade,taxa_recuperacao,obito_1000_habitantes,casos_1000_habitantes
data,,,,,,,,,,
2020-03-23,ABOLICAO,0.0,0.0,1.0,1.0,11356.0,0.000000,100.000000,0.000000,0.088059
2020-03-30,ABOLICAO,0.0,1.0,2.0,3.0,11356.0,33.333333,66.666667,0.088059,0.264178
2020-04-06,ABOLICAO,0.0,2.0,9.0,11.0,11356.0,18.181818,81.818182,0.176118,0.968651
2020-04-13,ABOLICAO,0.0,4.0,3.0,7.0,11356.0,57.142857,42.857143,0.352237,0.616414
2020-04-20,ABOLICAO,0.0,5.0,5.0,10.0,11356.0,50.000000,50.000000,0.440296,0.880592


In [35]:
#bairros = df_bairro_top10.index
bairros = df_bairro.sort_values('total', ascending=False)[:10].index
df_bairro_1 = df_bairro_diario[df_bairro_diario['bairro'].isin(bairros)]

desde = df_bairro_1.index.max().strftime("%d/%m")
fig = px.line(df_bairro_1, 
              width=1280,
              x=df_bairro_1.index, 
              y=df_bairro_1["obito"],
              line_shape="spline",
              color=df_bairro_1["bairro"],
              template="simple_white",
              title='Evolução dos óbitos nos bairros com mais casos de COVID-19')

fig.update_yaxes(showticklabels=True, visible=False)
fig.update_xaxes(showticklabels=True, visible=True, title='')

fig.show()
fig.write_html(os.path.join(save_path, 'evolucao-casos-obitos-bairros-top10.html'))

## Letalidade x Recuperação geral desde o início da pandemia na cidade

In [36]:
df_dia  = df.groupby(['dt_notificacao','evolucao']).classificacao.count().reset_index().\
pivot_table(index=['dt_notificacao'],columns=['evolucao']).reset_index()

df_dia.columns = ['data','ativo','obito','recuperado']
df_dia['recuperado'] = df_dia.recuperado.fillna(0)
df_dia['ativo'] = df_dia.ativo.fillna(0)
df_dia['obito'] = df_dia.obito.fillna(0)
df_dia['total'] = df_dia.ativo + df_dia.recuperado + df_dia.obito

df_dia['obito_acumulado'] = df_dia.obito.cumsum()
df_dia['recuperado_acumulado'] = df_dia.recuperado.cumsum()
df_dia['total_acumulado'] = df_dia.total.cumsum()
df_dia['aumento'] = df_dia.obito_acumulado - df_dia.obito_acumulado.shift(+1)
df_dia['aceleracao'] = df_dia.aumento - df_dia.aumento.shift(+1)

df_dia['taxa_letalidade'] = (df_dia.obito_acumulado / df_dia.total_acumulado) * 100
df_dia['taxa_recuperacao'] =(df_dia.recuperado_acumulado / df_dia.total_acumulado) * 100

print("Taxa de letalidade em {}: {}".format(ate, (df_dia.sum().obito / df_dia.sum().total) * 100))
print("Taxa de recuperação em {}: {}".format(ate, (df_dia.sum().recuperado / df_dia.sum().total) * 100))
df_dia = df_dia.set_index('data')

Taxa de letalidade em 03/29/21: 8.991001853216336
Taxa de recuperação em 03/29/21: 85.31359546296916


In [37]:
df_dia.tail()

,ativo,obito,recuperado,total,obito_acumulado,recuperado_acumulado,total_acumulado,aumento,aceleracao,taxa_letalidade,taxa_recuperacao
data,,,,,,,,,,,
2021-03-25,403.0,9.0,133.0,545.0,20126.0,190846.0,223115.0,9.0,-30.0,9.020460,85.537055
2021-03-26,290.0,7.0,132.0,429.0,20133.0,190978.0,223544.0,7.0,-2.0,9.006281,85.431951
2021-03-27,180.0,0.0,48.0,228.0,20133.0,191026.0,223772.0,0.0,-7.0,8.997104,85.366355
2021-03-28,140.0,1.0,20.0,161.0,20134.0,191046.0,223933.0,1.0,1.0,8.991082,85.313911
2021-03-29,1.0,0.0,1.0,2.0,20134.0,191047.0,223935.0,0.0,-1.0,8.991002,85.313595


## Total de casos e óbitos capital Rio de Janeiro

In [39]:
desde = df_dia.index.min().strftime("%d/%m")
fig = px.line(df_dia, 
              width=1280,
              x=df_dia.index, 
              y=["aumento"],
              line_shape="spline",
              template="simple_white",
              title='Evolução de casos e óbitos' )

fig.update_yaxes(showticklabels=True, visible=False)
fig.update_xaxes(showticklabels=True, visible=True, title='')

fig.show()
fig.write_html(os.path.join(save_path, 'aumento-casos.html'))

In [40]:
desde = df_dia.index.min().strftime("%d/%m")
fig = px.line(df_dia, 
              width=1280,
              x=df_dia.index, 
              y=["aceleracao"],
              line_shape="spline",
              template="simple_white",
              title='Evolução de casos e óbitos' )

fig.update_yaxes(showticklabels=True, visible=False)
fig.update_xaxes(showticklabels=True, visible=True, title='')

fig.show()
fig.write_html(os.path.join(save_path, 'aceleracao-casos.html'))

In [41]:
desde = df_dia.index.min().strftime("%d/%m")
fig = px.line(df_dia, 
              x=df_dia.index, 
              y=["taxa_letalidade"],
              line_shape="spline",
              template="xgridoff",
              title='Taxa de letalidade' )

fig.update_yaxes(showticklabels=True, visible=False)
fig.update_xaxes(showticklabels=True, visible=True, title='')

fig.show()
fig.write_html(os.path.join(save_path, 'taxa-letalidade.html'))

## Download files

In [59]:
from google.colab import files

In [62]:
for filename in os.listdir(save_path):
    if filename.endswith(".html"):
        files.download(os.path.join(save_path, filename)) 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>